Домашнее задание основано на этой тетрадке - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/spellcheck.ipynb

## Домашнее задание 2. Исправление оЧепяток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance

### Задание:

В тетрадке есть функция get_closest_match_vec, которая находит варианты исправлений по символьному векторному представлению слов. Однако она очень плохо учитывает порядок символов в слове.

Расстояние Левенштейна находит более точные исправления, но работает долго.

#### Задача:

In [3]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}
vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [4]:
WORDS

Counter({'в': 67679,
         'выставочном': 7,
         'центре': 189,
         'абу-даби': 17,
         'недавно': 308,
         'состоялась': 68,
         '3-я': 11,
         'выставка': 45,
         'и': 55933,
         'конференция': 41,
         'области': 647,
         'беспилотных': 95,
         'систем': 291,
         'umex-2018': 15,
         'на': 27860,
         'мероприятии': 21,
         'были': 1930,
         'традиционно': 88,
         'представлены': 85,
         'беспилотные': 28,
         'авиационные': 24,
         'системы': 698,
         'различного': 27,
         'типа': 238,
         'класса': 148,
         'наземные': 15,
         'роботизированные': 10,
         'комплексы': 30,
         'а': 9696,
         'также': 2393,
         'безэкипажные': 7,
         'катера': 12,
         'необитаемые': 5,
         'подводные': 29,
         'аппараты': 37,
         'ведущую': 16,
         'роль': 402,
         'здесь': 924,
         'конечно': 756,
         'играли': 

1) совместить get_closest_match_vec и левенштейна.
Новая функция должна находить несколько (3-10) вариантов по векторной близости, а затем для каждого из вариантов рассчитывать расстояние левенштейна и выдавать лучший.

In [5]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    return [id2word[top] for top in topn]

In [6]:
def get_closest_hybrid_match(text, X, vec, TOPN=3, metric=textdistance.levenshtein):
    candidates = get_closest_match_vec(text, X, vec, TOPN = TOPN)
    similarities = Counter()
    for word in candidates:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(1)[0][0]

In [7]:
%%time
TOPN = 3
print(get_closest_hybrid_match('опофеоз', X, vec, TOPN))

апофеоз
Wall time: 80 ms


In [8]:
%%time
TOPN = 10
print(get_closest_hybrid_match('опофеоз', X, vec, TOPN))

апофеоз
Wall time: 76.9 ms


2) Проверить качество работы такой функции на данных соревнования Диалога.

In [9]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [10]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [11]:
correct = 0
total = 0
mistake_word_pairs = []
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            mistake_word_pairs.append(pair)
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.5333333333333333
10
0.7615384615384615
20
0.7748917748917749
30
0.7702020202020202
40
0.7680311890838206
50
0.7761674718196457
60
0.7666666666666667
70
0.7747524752475248
80
0.7736450584484591
90
0.7740667976424361
100
0.7724014336917563
110
0.7766582703610412
120
0.7811291569992266
130
0.7794871794871795
140
0.7778537252221462
150
0.7870722433460076
160
0.7871462264150944
170
0.7883211678832117
180
0.7868589743589743
190
0.7890781563126252
200
0.7922077922077922
210
0.7944646098003629
220
0.794344473007712
230
0.7963039014373716
240
0.8009277155005798
250
0.7980168931325744
260
0.7995075624340485
270
0.7974983096686951
280
0.7992900935785737
290
0.7972006220839813
300
0.7992788461538461
310
0.8002932551319648
320
0.8003969379075702
330
0.7997790665561999
340
0.7992995689655172
350
0.7969118031928815
360
0.7977698935631019
370
0.7983130736789879
380
0.7990349819059107
390
0.7996210326859309
400
0.7998144282069125
410
0.7991433724075744
420
0.7985388532211645
430
0.7983211364614722


In [12]:
# запихиваем все в одну большую общую функцию, чтобы легче жилось
def main_function (correct, total, mistake_word_pairs):
    correct = 0
    total = 0
    mistake_word_pairs = []
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            predicted = get_closest_hybrid_match(pair[1], X, vec)
            if predicted == pair[0]:
                correct += 1
            else:
                mistake_word_pairs.append(pair)
            total += 1
        if not i % 10:
            print(i)
            print(correct/total)

3) Посмотреть на ошибки

In [13]:
mistake_word_pairs[:30]

[('симпатичнейшее', 'симпатичнейшое'),
 ('шпионское', 'шпионское'),
 ('гламурный', 'гламурный'),
 ('бонда', 'бонда'),
 ('superheadz', 'superheadz'),
 ('clap', 'clap'),
 ('camera', 'camera'),
 ('получатся', 'полчатся'),
 ('язычки', 'язычки'),
 ('все', 'все'),
 ('очень', 'оччччень'),
 ('милые', 'милые'),
 ('насчет', 'нащщот'),
 ('чавеса', 'чавеса'),
 ('на', 'на'),
 ('попавшим', 'попавшим'),
 ('аварийно-спасательных', 'аварийно-спасательных'),
 ('напрасно', 'нарасно'),
 ('на', 'на'),
 ('в', 'вобщем'),
 ('общем', 'как'),
 ('как', 'вы'),
 ('вы', 'знаете'),
 ('знаете', 'из'),
 ('из', 'моего'),
 ('моего', 'не'),
 ('недавнего', 'давнего'),
 ('на', 'на'),
 ('пропажу', 'пропажу'),
 ('на', 'на')]

In [14]:
predicted = get_closest_hybrid_match('все', X, vec)
print(predicted)

есв


4*) Попробовать улучшить качество любым способом (необязательное задание, которое даст вам +1 доп балл) Несколько идей: взять другой корпус, по которому строится словарь правильных слов, использовать буквенные нграмы (ngram_range=(1,4) в CountVectorizer), добавлять в словарь только слова, встретившиеся >n раз, подключить pymorphy или mystem, чтобы выдавать исправление только такой же части речи.

In [15]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}
vec = TfidfVectorizer(analyzer='char', ngram_range=(1,4)) # используем буквенные нграмы (ngram_range=(1,4) в CountVectorizer
X = vec.fit_transform(vocab)

In [16]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [17]:
X

<137738x85854 sparse matrix of type '<class 'numpy.float64'>'
	with 4034906 stored elements in Compressed Sparse Row format>

In [18]:
d = dict(WORDS)

In [19]:
d

{'в': 67679,
 'выставочном': 7,
 'центре': 189,
 'абу-даби': 17,
 'недавно': 308,
 'состоялась': 68,
 '3-я': 11,
 'выставка': 45,
 'и': 55933,
 'конференция': 41,
 'области': 647,
 'беспилотных': 95,
 'систем': 291,
 'umex-2018': 15,
 'на': 27860,
 'мероприятии': 21,
 'были': 1930,
 'традиционно': 88,
 'представлены': 85,
 'беспилотные': 28,
 'авиационные': 24,
 'системы': 698,
 'различного': 27,
 'типа': 238,
 'класса': 148,
 'наземные': 15,
 'роботизированные': 10,
 'комплексы': 30,
 'а': 9696,
 'также': 2393,
 'безэкипажные': 7,
 'катера': 12,
 'необитаемые': 5,
 'подводные': 29,
 'аппараты': 37,
 'ведущую': 16,
 'роль': 402,
 'здесь': 924,
 'конечно': 756,
 'играли': 23,
 'местные': 105,
 'компании': 618,
 'финансовые': 74,
 'фонды': 24,
 'которые': 2566,
 'ведут': 91,
 'отбор': 36,
 'продвижение': 29,
 'различных': 257,
 'как': 8958,
 'правило': 135,
 'зарубежных': 97,
 'проектов': 310,
 'регионе': 261,
 'так': 3944,
 'макет': 29,
 'беспилотного': 23,
 'летательного': 18,
 'аппара

In [22]:
vocab = list([x[0] for x in d if x[1] > 1]) # добавлять в словарь только слова, встретившиеся >n раз

IndexError: string index out of range

Берем другой корпус. В нашем случае - это Открытый корпус русского языка, который можно скачать по следующей ссылке: http://opencorpora.org/dict.php

In [23]:
better_corpus = [sent.split() for sent in open('dict.opcorpora.txt', encoding='utf8').read().splitlines()]

In [24]:
better_corpus[:30]

[['1'],
 ['ЁЖ', 'NOUN,anim,masc', 'sing,nomn'],
 ['ЕЖА', 'NOUN,anim,masc', 'sing,gent'],
 ['ЕЖУ', 'NOUN,anim,masc', 'sing,datv'],
 ['ЕЖА', 'NOUN,anim,masc', 'sing,accs'],
 ['ЕЖОМ', 'NOUN,anim,masc', 'sing,ablt'],
 ['ЕЖЕ', 'NOUN,anim,masc', 'sing,loct'],
 ['ЕЖИ', 'NOUN,anim,masc', 'plur,nomn'],
 ['ЕЖЕЙ', 'NOUN,anim,masc', 'plur,gent'],
 ['ЕЖАМ', 'NOUN,anim,masc', 'plur,datv'],
 ['ЕЖЕЙ', 'NOUN,anim,masc', 'plur,accs'],
 ['ЕЖАМИ', 'NOUN,anim,masc', 'plur,ablt'],
 ['ЕЖАХ', 'NOUN,anim,masc', 'plur,loct'],
 [],
 ['2'],
 ['ЁЖ', 'NOUN,inan,masc', 'sing,nomn'],
 ['ЕЖА', 'NOUN,inan,masc', 'sing,gent'],
 ['ЕЖУ', 'NOUN,inan,masc', 'sing,datv'],
 ['ЁЖ', 'NOUN,inan,masc', 'sing,accs'],
 ['ЕЖОМ', 'NOUN,inan,masc', 'sing,ablt'],
 ['ЕЖЕ', 'NOUN,inan,masc', 'sing,loct'],
 ['ЕЖИ', 'NOUN,inan,masc', 'plur,nomn'],
 ['ЕЖЕЙ', 'NOUN,inan,masc', 'plur,gent'],
 ['ЕЖАМ', 'NOUN,inan,masc', 'plur,datv'],
 ['ЕЖИ', 'NOUN,inan,masc', 'plur,accs'],
 ['ЕЖАМИ', 'NOUN,inan,masc', 'plur,ablt'],
 ['ЕЖАХ', 'NOUN,inan,masc',